## Project B：产品关联分析

In [1]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd 

# 数据加载
data = pd.read_csv('./订单表.csv',encoding="gbk")
order_data = data[['产品名称','客户ID','订单日期']] 
order_data

,产品名称,客户ID,订单日期
0,车胎和内胎,12411,2016/1/1
1,车胎和内胎,16720,2016/1/2
2,车胎和内胎,25967,2016/1/2
3,车胎和内胎,23689,2016/1/5
4,车胎和内胎,12978,2016/1/6
...,...,...,...
60393,车胎和内胎,25177,2016/7/28
60394,车胎和内胎,25066,2016/7/29
60395,车胎和内胎,21364,2016/7/29
60396,车胎和内胎,13423,2016/7/30


In [2]:
#数据清洗
# 去掉none项
order_data = order_data.drop(order_data[order_data.产品名称 == 'none'].index)   #删除行：df.drop([行的索引]，inplace=TRUE/FALSE);删除列：df.drop(['col1',col2],axis = 1,inplace = )

In [3]:
#grouped = data.groupby(['Transaction','Item'])['Item'].count()
#grouped
#转化成适合One hot编码的样式，通过行列透视实现
hot_encoded_df=order_data.groupby(['客户ID','订单日期','产品名称'])['产品名称'].count().unstack().reset_index().fillna(0).set_index('客户ID')   #unstack()将行中数据透视到列;stack()将列中数据透视到行；也可用pivot实现
hot_encoded_df

产品名称,订单日期,公路自行车,头盔,山地自行车,帽子,手套,挂架,挡泥板,旅行自行车,水壶和水壶架,水袋背包,清洁剂,短裤,背心,自行车支架,袜子,车胎和内胎,骑行服
客户ID,,,,,,,,,,,,,,,,,,
11000,2013/7/22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11000,2015/11/4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
11000,2015/7/22,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11001,2013/7/18,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11001,2015/7/20,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,2015/3/8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29480,2016/1/18,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29481,2014/2/13,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
hot_encoded_df.drop('订单日期',axis = 1,inplace = True)

In [5]:
#print(hot_encoded_df)
#one hot编码函数转换
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
#函数encode用在每一个数据上，实现转换    
hot_encoded_df = hot_encoded_df.applymap(encode_units)
#使用mlxtend工具箱的apriori关联方法
frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
itemsets_sort = frequent_itemsets.sort_values(by = 'support',ascending = False)
# 根据频繁项集计算关联规则，设置最小提升度为2
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules_sort = rules.sort_values(by = 'lift',ascending = False)

print('频繁项集:',frequent_itemsets)
print('关联规则：',rules)

频繁项集:      support         itemsets
0   0.292128          (公路自行车)
1   0.233145             (头盔)
2   0.179955          (山地自行车)
3   0.079296             (帽子)
4   0.051778             (手套)
5   0.076798            (挡泥板)
6   0.078463          (旅行自行车)
7   0.172605         (水壶和水壶架)
8   0.026541           (水袋背包)
9   0.032877            (清洁剂)
10  0.036896             (短裤)
11  0.020349             (背心)
12  0.020566             (袜子)
13  0.356434          (车胎和内胎)
14  0.120646            (骑行服)
15  0.048990      (头盔, 公路自行车)
16  0.035701  (水壶和水壶架, 公路自行车)
17  0.032913   (车胎和内胎, 公路自行车)
18  0.029039     (骑行服, 公路自行车)
19  0.033819      (头盔, 山地自行车)
20  0.021797         (头盔, 手套)
21  0.020820        (挡泥板, 头盔)
22  0.032334      (旅行自行车, 头盔)
23  0.043559     (头盔, 水壶和水壶架)
24  0.100261      (头盔, 车胎和内胎)
25  0.025129        (头盔, 骑行服)
26  0.032515     (挡泥板, 山地自行车)
27  0.034325  (山地自行车, 水壶和水壶架)
28  0.032950   (车胎和内胎, 山地自行车)
29  0.023173     (帽子, 水壶和水壶架)
30  0.022341        (帽子, 骑行服)
31  0.020168      (车胎和内胎, 手套)
32  